In [ ]:
#Find all place names <placeName> in <body> that match the list  <listPlace> in the header. Then, fetch the place xml:id and some other useful elements.
#From only Sloane catnums
#Ignore index

In [ ]:
from lxml import etree
import re
import csv
import pandas as pd

In [ ]:
#to keep all namespaces correctly
#import re
#import xml.etree.ElementTree as ET

#ET.register_namespace('',"http://www.tei-c.org/ns/1.0")
#ET.register_namespace('ea',"http://www.enlightenmentarchitectures.org")
#mytree = ET.parse('/Users/deborahleem/Documents/1_PhD/SloaneDBwork19/Miscellanies5Ah_MASTER_fixed_whitespaceDL-KS_050321 (1).xml')

path = '/Users/deborahleem/Documents/1_PhD/XMLSloane2021/unicode_modified_misc5_persname_cleaned_up.xml'
doc = etree.parse(open(path))
mytree = doc.getroot()
root = mytree

In [ ]:
def remove_element(el):
    parent = el.getparent()
    if el.tail:
        prev = el.getprevious()
        if prev:
            prev.tail = (prev.tail or '') + el.tail
        else:
            parent.text = (parent.text or '') + el.tail
    parent.remove(el)

# Remove every <add rend="del"> and <add rend="pencil"> from document before the rest of analysis
for e in list(root.iterfind('.//{*}add')):
    if ('rend' in e.attrib) and (e.attrib['rend'] == 'del' or e.attrib['rend'] == 'pencil'):
        #print('removing', etree.tostring(e, pretty_print=True))
        if e.text == '-1861' or e.text == '2094.' or e.text == '-382.':
            continue
        remove_element(e)

In [ ]:
#From only Sloane catnums. So the following need to be excluded

part_excluded = {
  "EPF108491027v": ["2108","2109","2110","2111"], # "Miscellanies"
  "EPF108491177v": ["1","2"], # "Antiquities"    
}

excluded = {
  "EPF108491178v": ["1","2","3","4"], # "Antiquities"
  "EPF108491179v": ["4","5","6","7","8","1"], # "Antiquities"
  "EPF108491180v": ["2","3"], # "Antiquities"
  "EPF108491181v": ["4","5"], # "Antiquities"
  "EPF108491182v": ["1","2","3","4","5"], # "Antiquities"
  "EPF108491183v": ["6","7","8","9","10","11","12"], # "Antiquities"
  "EPF108491184v": ["13","14","1","2","3","4","5","1"], # "Antiquities"
  "EPF108491186v": ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"], # "Antiquities"
  "EPF108491187v": ["20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39"], # "Antiquities"
  "EPF108491188v": ["40","41","42","43","44","45","46","47","48","1","2","3","4","5","6","7","8","9"], # "Antiquities"
  "EPF108491189v": ["10","11","12","1","2","3","4","5","6"], # "Antiquities"
  "EPF108491190v": ["7","8","9","10","11"], # "Antiquities"
}

In [ ]:
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return ''.join(label.itertext()).strip()
            return
        element = element.getparent()

In [ ]:
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()

In [ ]:
def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

In [ ]:
#which placeName goes with which ID

#viaf map shows viaf number with the corresponding person name
# This has nothing to do with the body text
idmap = {}
variant_id_map = {}
# go through master list place names
for x in mytree.findall('.//{http://www.tei-c.org/ns/1.0}listPlace/{http://www.tei-c.org/ns/1.0}place'):
    id = x.attrib['{http://www.w3.org/XML/1998/namespace}id']
    # do find all on x object for placeName
    for name in x.findall('{http://www.tei-c.org/ns/1.0}placeName'):
        text = name.text
        # create a mapping of name to place_id        
        idmap[text.lower()] = '#' + id
        variant_id_map.setdefault(text.lower(), []).append(id)
        
#         idno = x.find("{*}idno")
#         # if idno is found add to viafmap
#         if idno is not None:
#             viafmap[text] = idno.text


In [ ]:
idmap

In [ ]:
# find mappings where there are multiple ids linked to a place name
{k:v for (k,v) in variant_id_map.items() if len(v) > 1}

In [ ]:
def lookup_place(name, x):
    """Lookup in our dictionary created from the master list to obtain place name ID"""
    #     Use the value from our dictionary variant_id_map
#     if name == 'Jamaica':
#         import pdb; pdb.set_trace()
    try:
        val = str(idmap[name.lower()])
#         val = val.replace('[', '').replace(']', '').replace('"', '').replace("'", '')
        print(name)
        print('Adding in:' + val)
#         return val
#         x.set('key', val)
        x.set('ref', val)
    except:
#           print('Issue adding in: ' + name)
        print('Issue adding in: ' + name + '\ncat num: ' + str('') + '\npage: ' + str('') + '\n')

In [ ]:
# We must reload the xml root.This is a duplicate of the first cell.
# This is because we dont want to remove things like add_rend.
# The lookups above are still in memory.
path = '/Users/deborahleem/Documents/1_PhD/XMLSloane2021/unicode_modified_misc5_persname_cleaned_up.xml'
doc = etree.parse(open(path))
mytree = doc.getroot()
root = mytree

In [ ]:
#ignore non-Sloane catnums and index <label type="index">. End of index </div2>
place_csv_data = []

# labels = set()

# for div3 in root.iter("{*}div3"):
for x in mytree.findall('.//{http://www.tei-c.org/ns/1.0}div3//{http://www.tei-c.org/ns/1.0}placeName'):
    name = re.sub(r'\s+', " ", ''.join(x.itertext()))
    name = re.sub(r'^\s+', '', name)
    name = re.sub(r'\s+$', '', name)   
    lookup_place(name, x)
    
#     places = list(div3.iter('{*}placeName'))
#     catnums = list(div3.iter('{*}catnum'))
#     clean_catnums = [cleanup_catnum(''.join(catnum.itertext())) for catnum in catnums]
    
#     page_id = find_pb_xml_id(div3)
#     label = find_div2_label(div3)
    
# #     labels.add(label)
#     if label.startswith('Index'):
#         continue
    
#     if len(places) > 0:
#         if page_id in excluded:
#             continue

#         This is an individual row to be saved as a dict
#         res = {}

#         res["div2_label"] = label
#         div3_text = ''.join(div3.itertext())

#         catnum_res = []
        
#         Clean the catnums
#         for name in clean_catnums:
#             if not name:
#                 name = ''
#             catnum_res.append(name)
            
#         if not catnum_res:
#             catnum_res = ['']
#         res["catnum"] = catnum_res
        
#         places_res = []
#         places_id_res = []
#         for place in places:
#             import pdb; pdb.set_trace()
#     name = re.sub(r'\s+', " ", ''.join(place.itertext()))

#             places_id_res.append(place_id)
            
#         res["place"] = places_res
#         res["place_ids"] = places_id_res
        
#         place_csv_data.append(res)

# print(labels)

In [ ]:
# save down the xml
doc.write("output_person_place_misc.xml", encoding='utf-8')

In [ ]:
from pprint import pprint as pp
# Place names from body
pp(place_csv_data)

In [ ]:
# def find_catnums_in_div3(div3):
#     return [cleanup_catnum(''.join(cn.itertext())) for cn in div3.findall("{*}catnum")]

# #matching names and inserting ID into persName ref
# matches = {False: 0, True: 0}

# data = []
# # This is main body
# for x in mytree.findall('.//{http://www.tei-c.org/ns/1.0}div3//{http://www.tei-c.org/ns/1.0}persName'):
#     # All text inside persName tag
#     name = re.sub(r'\s+', " ", ''.join(x.itertext()))
#     name = cleanup_person_name(name)
    
#     # A few more rules
# #     if name not in idmap:
# #         print("not match", name)

# #     if name in idmap:
#     div3 = find_enclosing_div3(x)
#     div3_text = ''.join(div3.itertext()).strip()
#     catnums = find_catnums_in_div3(div3)
#     label = find_div2_label(x)
#     if label.startswith('Index'):
#         continue
#     page = find_pb_xml_id(x)
# #etree.tounicode(div3) to get XML
#     # Modification to add in new attr to xml based on new requirement
#     lookup_name(name, x, catnums, page)
#     row = [name, idmap.get(name, None), viafmap.get(name, None), catnums, label, page, div3_text, etree.tounicode(div3)]
#     data.append(row)

# #         matches[name in idmap] += 1
# #     if name in idmap:
# #         x.attrib['ref'] = idmap[name]
        
# #print matches and count non-matches    
# # print(matches)

# '''
# people_match_nonmatch_text.csv' gives the names that match from the header inc normalised names
# and also the ones do not have their names in the header
# '''
# with open('people_match_nonmatch_text.csv', 'w') as file:
#     writer = csv.writer(file)
#     for row in data:
#         writer.writerow(row)

In [ ]:
# Create the dataframe and populate based on the dictionary

df = pd.DataFrame({'div2_label': [], 'place': [], 'place_ids': [], 'catnum': []})
for row in place_csv_data:
    df = df.append({'div2_label': row['div2_label'], 'place': str(row['place']), 'place_ids': str(row['place_ids']), 'catnum': str(row['catnum'])}, ignore_index=True)
df.head()

In [ ]:
df[df.place_ids != '[None]'].to_csv('places_with_found_name_from_master_list.csv')


In [ ]:
len(df[df.place_ids != '[None]'])

In [ ]:
df[df.place_ids == '[None]'].to_csv('places_with_missing_name_from_master_list.csv')

In [ ]:
len(df[df.place_ids == '[None]'])

In [ ]:
# number of records
len(df)

In [ ]:
variant_id_map

In [ ]:
#total number of place name (count every place name element)
len()

In [ ]:
#matching place names and inserting ID into placeName ref. eg. <placeName ref="#Jerusalem">Jerusalem</placeName>
#There are existing ref already but not good enough.

In [ ]:
#print the place names that do not match the list in the header along with their catnum, div2, and page number